<a href="https://colab.research.google.com/github/robert-lamprecht/Computational-Neuroscience/blob/main/CompNeuroProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Libraries
!pip install numpy scipy scikit-learn --quiet
# !pip install --upgrade pandas numpy

# Import libraries
import numpy as np
import scipy
import scipy.io #enables uploading of .mat files
import matplotlib.pyplot as plt
# from sklearn.feature_selection import mutual_info_classif #for MI analysis
import math
from scipy import stats as st
from scipy import signal
from scipy import interpolate
from scipy.io import loadmat
print("Libraries loaded successfully!")


Libraries loaded successfully!


In [ ]:
# # Load .mat files in in Python
# import scipy.io
# dat = scipy.io.loadmat('t5.2019.05.08_singleLetters.mat') #load single letters file
# mouseTemplate = scipy.io.loadmat('computerMouseTemplates.mat') #load handwriting file


FileNotFoundError: [Errno 2] No such file or directory: 't5.2019.05.08_singleLetters.mat'

In [ ]:
# Helper Function
def gaussSmooth_fast(timeSeries, width):
    if width == 0:
        return timeSeries

    wingSize = math.ceil(width * 5)
    # Range from -wingSize to wingSize inclusive
    x_range = np.arange(-wingSize, wingSize + 1)
    gKernel = st.norm.pdf(x_range, 0, width)
    # In Python, we don't need the conjugate transpose (conj().T)
    # since we're working with real values

    normFactor = np.cumsum(gKernel)
    test = np.vstack((timeSeries, np.zeros((len(gKernel)-1, timeSeries.shape[1]))))

    # Apply the filter
    Y = signal.lfilter(gKernel, [1], test)

    # Division operations (equivalent of bsxfun in MATLAB)
    Y[0:len(gKernel)-1, :] = Y[0:len(gKernel)-1, :] / normFactor[0:len(normFactor)-1, np.newaxis]
    Y[-(len(gKernel)-1):, :] = Y[-(len(gKernel)-1):, :] / np.flip(normFactor[0:len(normFactor)-1, np.newaxis], axis=0)

    # Extract the relevant part (equivalent to the last line in MATLAB)
    midpoint = (len(gKernel) - 1) // 2
    Y = Y[midpoint:-(len(gKernel)-1-midpoint), :]

    return Y

In [ ]:
# Helper Function
def tsne_warp_dist(d1, d2_mat, n_time_bins_per_trial):
    """
    Parameters:
    -----------
    d1 : numpy.ndarray
        A 1 x N vector representing a single data point that has been
        'unrolled' from a matrix (T x D) into a vector (1 x TD), where T is
        the number of time bins and D is the number of neural dimensions.

    d2_mat : numpy.ndarray
        An M x N matrix, where each row is a data point.

    n_time_bins_per_trial : int
        Specifies how many time bins (T) are included in each data point.
        The number of neural dimensions is then D = N/T.

    Returns:
    --------
    warp_dist : numpy.ndarray
        An M x 1 vector representing the distance between d1 and each row of d2.
    """
    # affineWarps is a vector of alpha values to consider
    affine_warps = np.linspace(0.7, 1.42, 15)

    # infer the number of neural dimensions per data point
    n_neural_dim = len(d1) // n_time_bins_per_trial

    # reshape d1 into a T x D matrix
    d1 = d1.reshape(n_time_bins_per_trial, n_neural_dim)

    # eDist represents the euclidean distance between d1 and all rows of d2
    # for each alpha
    e_dist = np.zeros((d2_mat.shape[0], len(affine_warps)))

    # now we fill in eDist one entry at a time
    for a in range(len(affine_warps)):
        # linearly warp d1 using this alpha
        x_orig = np.arange(1, d1.shape[0] + 1)
        x_interp = np.linspace(1, d1.shape[0], int(affine_warps[a] * d1.shape[0]))
        d1_interp = interpolate.interp1d(x_orig, d1, axis=0)(x_interp)

        # compute the euclidean distance between the warped d1 and all points in d2
        for row_idx in range(d2_mat.shape[0]):
            # reshape d2 into a T x D matrix
            d2 = d2_mat[row_idx, :].reshape(n_time_bins_per_trial, n_neural_dim)

            # compute the euclidean distance, taking care to compute only
            # over the relevant time points
            if affine_warps[a] > 1:
                df = d1_interp[:d1.shape[0], :] - d2
            else:
                df = d1_interp - d2[:d1_interp.shape[0], :]

            e_dist[row_idx, a] = np.mean(df**2)

    # the warp distance is defined as the minimum distance over all the
    # alphas, which we take here
    warp_dist = np.min(e_dist, axis=1)

    return warp_dist

In [ ]:
# to load files from google drive directly, if have shitty laptop
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Time Warping & PCA
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import pandas as pd

# Note: the filepath i use here is unique to my google drive, i couldnt figure out how to get
# the google drive mount to work on shared folders so i copied the mat files to my drive for access
dat = scipy.io.loadmat('/content/drive/MyDrive/Emory_Year_2/COMP NEURO/t5.2019.05.08_singleLetters.mat')
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
           'greaterThan','comma','apostrophe','tilde','questionMark']

# Normalize the neural activity by blockwise z-scoring
for letter in letters:
    norm_cube = np.array(dat[f'neuralActivityCube_{letter}'], dtype=np.float32)

    t_idx = np.arange(3)
    for y in range(9):
        mn = np.zeros((3, 1, 192))
        mn[0, 0, :] = dat['meansPerBlock'][y, :]
        mn[1, 0, :] = dat['meansPerBlock'][y, :]
        mn[2, 0, :] = dat['meansPerBlock'][y, :]

        sd = np.zeros((1, 1, 192))
        sd[0, 0, :] = dat['stdAcrossAllData']

        norm_cube[t_idx, :, :] -= mn
        norm_cube[t_idx, :, :] /= sd
        t_idx += 3

    dat[f'neuralActivityCube_{letter}'] = norm_cube

# Compute trial-averaged activity for each character
all_data = np.zeros((2000, 27264))  # You may want to adjust this size
all_spatial = np.zeros((200000, 192))  # You may want to adjust this size
all_labels = np.zeros(2000, dtype=int)
all_avg = []
c_idx = 0
spatial_idx = 0

for f, letter in enumerate(letters):
    letter_cube = np.array(dat[f'neuralActivityCube_{letter}'])
    for x in range(letter_cube.shape[0]):
        # Apply Gaussian smoothing
        row = gaussian_filter1d(letter_cube[x, 60:, :], 3, axis=0)
        all_data[c_idx, :] = row.flatten()
        all_labels[c_idx] = f
        c_idx += 1

        # Store spatial data
        new_chunk = gaussian_filter1d(letter_cube[x, 60:, :], 5, axis=0)
        all_spatial[spatial_idx:(spatial_idx + new_chunk.shape[0]), :] = new_chunk
        spatial_idx += new_chunk.shape[0]

    # Averaged letter activity
    avg_let = np.mean(letter_cube, axis=0)
    avg_let = gaussian_filter1d(avg_let[60:, :], 5, axis=0)
    all_avg.append(avg_let)

# Apply PCA to the trial-averaged activity data
all_avg = np.vstack(all_avg)
pca = PCA(n_components=15)
all_data_pca = pca.fit_transform(all_avg)


ValueError: could not broadcast input array from shape (27072,) into shape (27264,)

In [ ]:
#t-SNE and KNN

#list of letters
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w','x','y','z', 'greaterThan','comma','apostrophe','tilde','questionMark']

#normalize the neural activity by blockwise z-scoring
for letter in letters:
    normCube = np.float32(dat['neuralActivityCube' + letter])



neuralActivityCubea
neuralActivityCubeb
neuralActivityCubec
neuralActivityCubed
neuralActivityCubee
neuralActivityCubef
neuralActivityCubeg
neuralActivityCubeh
neuralActivityCubei
neuralActivityCubej
neuralActivityCubek
neuralActivityCubel
neuralActivityCubem
neuralActivityCuben
neuralActivityCubeo
neuralActivityCubep
neuralActivityCubeq
neuralActivityCuber
neuralActivityCubes
neuralActivityCubet
neuralActivityCubeu
neuralActivityCubev
neuralActivityCubew
neuralActivityCubex
neuralActivityCubey
neuralActivityCubez
neuralActivityCubegreaterThan
neuralActivityCubecomma
neuralActivityCubeapostrophe
neuralActivityCubetilde
neuralActivityCubequestionMark


In [ ]:
# Helper Function

def getHandwritingCharacterDefinitions():
  """
  Returns a dictionary with entries that define the names of each character, its length, and where the pen tip begins.

  Returns:
      charDef(dict)
  """

  charDef = {}

  # Define the list of all 31 characters and their names
  charDef['charList'] = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                'greaterThan','comma','apostrophe','tilde','questionMark']
  charDef['charListAbbr'] = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                '>',',',"'",'~','?']

  # Define the length of each character (in # of 10 ms bins) to use for each template.
  # These were hand-defined based on visual inspection of the reconstructed pen trajectories.
  charDef['charLen'] = np.array([99, 91, 70, 104, 98, 125, 110, 104, 79, 92, 127, 68, 132, 90,
                        84, 113, 104, 74, 86, 110, 86, 83, 110, 103, 115, 100, 82, 77, 116, 71, 110]).astype(np.int32)

  # For each character, this defines the starting location of the pen tip (0 = bottom of the line, 1 = top)
  charDef['penStart'] = [0.25, 1, 0.5, 0.5, 0.25, 1.0, 0.25, 1.0, 0.5, 0.5, 1, 1, 0.5, 0.5, 0.25, 0.5, 0.25, 0.5, 0.5, 1,
           0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 1, 0.5, 1]

  # Dictionary to convert string representation to character index
  charDef['strToCharIdx'] = {}
  for x in range(len(charDef['charListAbbr'])):
    charDef['strToCharIdx'][charDef['charListAbbr'][x]] = x

  return charDef


In [ ]:
#Time-warping the single letters data for linear decoding of pen tip velocities

# Install and load the updated time-warping package ("Piecewise Linear Time Warping")
!pip install git+https://github.com/ahwillia/affinewarp.git

import affinewarp as aw
import scipy.io
from scipy.ndimage import gaussian_filter1d
from affinewarp.piecewisewarp import PiecewiseWarping # Piecewise warping is the closest warping function to the TWPCA done in the paper
import os

dat = scipy.io.loadmat('/content/drive/My Drive/Comp Neuro Project/t5.2019.05.08_singleLetters.mat')

  Cloning https://github.com/ahwillia/affinewarp.git to /tmp/pip-req-build-o2sinm9t
  Running command git clone --filter=blob:none --quiet https://github.com/ahwillia/affinewarp.git /tmp/pip-req-build-o2sinm9t
  Resolved https://github.com/ahwillia/affinewarp.git to commit 23f9e643d2e74ad930ef283311c2c14c585eb6b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#Time-warping the single letters data cont'd

# defines the list of all 31 characters and what to call then
charDef = getHandwritingCharacterDefinitions()

# Pre-processing and Normalizing
# Because baseline firing rates drift over time, we normalize each electrode's firing rate by subtracting its mean firing rate within each block of data (re-centering it).
# We also divide by each electrode's standard deviation to normalize the units.

for char in charDef['charList']:
    neuralCube = dat['neuralActivity_' + char].astype(np.float64)

    # get the trials that belong to this character
    trlIdx = []
    for t in range(dat['characterCues'].shape[0]):
      if dat['characterCues'][t,0] == char:
        trlIdx.append(t)







In [ ]:
#reconstruct letter trajectories from neural activity

# Noticed that this script actually imports warped data from the "Step1_timeWarp" they did - will come back working on this Monday (4/14) afternoon

# save the letters
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v', 'w', 'x', 'y', 'z',
    'greaterThan', 'comma', 'apostrophe', 'tilde', 'questionMark']

# fix the 'o' and 'x' templates which don't match T5's writing style

## 1. rotation matrix for -90 degrees:
theta = -90 * np.pi / 180  # convert to radians
rot90 = np.array([[np.cos(theta), np.cos(0)],[np.sin(theta), np.sin(0)]])

## apply rotation to the first two columns of the 'o' template

mouseTemplates['o'][:, 0:2] = (rot90 @ mouseTemplates['o'][:, 0:2].T).T

## 2. set y = -x for indices 22 to 43
mouseTemplates['x'][21:43, 1] = -mouseTemplates['x'][21:43, 0]

## flip x and y for indices 47 to 68
mouseTemplates['x'][46:68, 0:2] = -mouseTemplates['x'][46:68, 0:2]

